In [20]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import pickle

In [1]:
!pip install tensorflow

In [21]:
## load trained model and scaler pickel and onehot

model=load_model("model.h5")

## load encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender= pickle.load(file)


with open('scaler.pkl', 'rb') as file:
    scaler= pickle.load(file)

print(type(label_encoder_gender))

<class 'sklearn.preprocessing._label.LabelEncoder'>


In [22]:
# Example of input data
input_data = {
    'CreditScore' : 400,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 50000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 30000
}

In [23]:
with open("one_hot_encoder_geo.pkl", "rb") as file:
    label_encoder_geo = pickle.load(file)

print(type(label_encoder_geo))


<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [24]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()



c:\Users\palla\Downloads\Self_Study\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [25]:
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=label_encoder_geo.get_feature_names_out(['Geography'])
)

In [26]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,400,France,Male,40,3,50000,2,1,1,30000


In [27]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,400,France,1,40,3,50000,2,1,1,30000


In [28]:
input_df=pd.concat([input_df.drop('Geography', axis=1),geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,400,1,40,3,50000,2,1,1,30000,1.0,0.0,0.0


In [29]:
##scaling data
input_scaler = scaler.transform(input_df)
input_scaler

array([[-2.62911178,  0.89091075,  0.12136034, -0.68041201, -0.44060459,
         0.84584804,  0.62670381,  0.968496  , -1.2638014 ,  1.00400803,
        -0.57427105, -0.58350885]])

In [30]:
## predict

prediction = model.predict(input_scaler)
prediction

1/1 [==============================] - 0s 126ms/step


array([[0.04127958]], dtype=float32)

In [31]:
prediction_proba = prediction[0][0]
prediction_proba

0.041279584

In [32]:
if prediction_proba > 0.5:
    print("Custmer is likely to CHURN")
else:
    print("Customer is not likely to CHURN.")

Customer is not likely to CHURN.
